# Método Triaxial de Discernimiento — Notebook 02  
## Matriz 3×N y evaluación completa con 15 criterios

Este notebook define la **matriz completa de evaluación** del Método Triaxial de Discernimiento (F–C–P):

- F — Fundamento (realidad / evidencia)
- C — Contexto (situación / fuerzas externas)
- P — Principio (coherencia ética y sistémica)

Aquí:

- Definimos los **15 criterios** (5 por eje).
- Los organizamos como una **matriz 3×N**.
- Calculamos los valores de eje F, C y P como promedio de sus criterios.
- Obtenemos el valor global de discernimiento **D** y la **acción sugerida**.

Este será el **núcleo conceptual por defecto** para futuras implementaciones (CLI, API, LLM, etc.).


In [2]:
from dataclasses import dataclass
from typing import List, Dict
import pandas as pd


In [3]:
@dataclass
class Criterion:
    axis: str          # 'F', 'C' o 'P'
    code: str          # F1, F2, ..., P5
    name: str          # Nombre corto
    question: str      # Pregunta / criterio operativo


# Definimos los 15 criterios (los mismos del cuadro de evaluación)
CRITERIA: List[Criterion] = [
    # FUNDAMENTO (F)
    Criterion("F", "F1", "Fuente confiable", 
              "La fuente es confiable / verificable"),
    Criterion("F", "F2", "Consistencia con hechos", 
              "La afirmación es consistente con hechos conocidos"),
    Criterion("F", "F3", "Funcionamiento real del mundo", 
              "Coincide con el funcionamiento real del mundo"),
    Criterion("F", "F4", "Sin suposiciones forzadas", 
              "No requiere suposiciones forzadas para sostenerse"),
    Criterion("F", "F5", "Sin contradicción fuerte", 
              "No contradice evidencia sólida"),

    # CONTEXTO (C)
    Criterion("C", "C1", "Contexto claro", 
              "Existe contexto claro y transparente"),
    Criterion("C", "C2", "Sin presión emocional/manipulación", 
              "No proviene de emoción, presión o manipulación"),
    Criterion("C", "C3", "Sin estigmatizar grupos", 
              "No generaliza ni estigmatiza grupos completos"),
    Criterion("C", "C4", "Sin agendas interesadas", 
              "Está aislada de agendas políticas o mediáticas interesadas"),
    Criterion("C", "C5", "Marco realista", 
              "Los datos o hechos están situados en un marco realista"),

    # PRINCIPIO (P)
    Criterion("P", "P1", "Menor entropía", 
              "Produce menor entropía (menos caos/confusión)"),
    Criterion("P", "P2", "Coherencia interna y externa", 
              "Fomenta coherencia interna y externa"),
    Criterion("P", "P3", "Respeto a la dignidad", 
              "Respeta la dignidad humana e institucional"),
    Criterion("P", "P4", "Sin odio ni división", 
              "No alimenta odio, división o violencia"),
    Criterion("P", "P5", "Acción responsable", 
              "Abre caminos de acción responsable"),
]

len(CRITERIA)


15

In [4]:
def criteria_template() -> pd.DataFrame:
    """
    Devuelve un DataFrame con los 15 criterios y una columna 'valor'
    vacía para ser llenada (0.0–1.0).
    """
    df = pd.DataFrame([c.__dict__ for c in CRITERIA])
    df["valor"] = None   # se llenará después
    return df


def decide_action(D: float) -> str:
    """
    Regla por defecto (misma lógica que el CLI):
    """
    if D > 0.80:
        return "Adoptar y, si es útil, compartir"
    elif 0.50 <= D <= 0.80:
        return "Considerar válido, complementar con más análisis"
    elif 0.20 <= D < 0.50:
        return "Cuestionar, corregir o contrastar fuentes"
    else:
        return "Descartar o denunciar si es dañino / falso"


def axis_average(df: pd.DataFrame, axis: str) -> float:
    """
    Promedia los valores de un eje ('F', 'C', 'P').
    Supone que la columna 'valor' ya está llena con números 0.0–1.0.
    """
    vals = df.loc[df["axis"] == axis, "valor"].astype(float)
    return float(vals.mean())


In [5]:
@dataclass
class TriaxialMatrixResult:
    claim: str
    F: float
    C: float
    P: float
    D: float
    action: str
    table: pd.DataFrame  # matriz completa de criterios


def evaluate_with_matrix(claim: str, scores: Dict[str, float]) -> TriaxialMatrixResult:
    """
    scores: diccionario { 'F1':0.3, 'F2':0.5, ..., 'P5':0.8 }
    """
    df = criteria_template()

    # Asignar valores a cada criterio a partir del diccionario
    df["valor"] = df["code"].map(scores)

    # Chequeo rápido por si falta alguno
    if df["valor"].isnull().any():
        missing = df.loc[df["valor"].isnull(), "code"].tolist()
        raise ValueError(f"Faltan valores para: {missing}")

    F_val = axis_average(df, "F")
    C_val = axis_average(df, "C")
    P_val = axis_average(df, "P")

    D_val = (F_val + C_val + P_val) / 3.0
    action = decide_action(D_val)

    return TriaxialMatrixResult(
        claim=claim,
        F=F_val,
        C=C_val,
        P=P_val,
        D=D_val,
        action=action,
        table=df
    )


In [6]:
claim_1 = "Los maestros son responsables del deterioro moral de los jóvenes."

scores_1 = {
    # FUNDAMENTO
    "F1": 0.3,  # fuente confiable
    "F2": 0.3,  # consistente con hechos conocidos
    "F3": 0.2,  # funcionamiento real del mundo
    "F4": 0.2,  # sin suposiciones forzadas
    "F5": 0.5,  # sin contradicción fuerte (no hay evidencia sólida a favor)

    # CONTEXTO
    "C1": 0.5,  # contexto claro
    "C2": 0.2,  # muy cargado de emoción/presión
    "C3": 0.0,  # estigmatiza a todo el grupo
    "C4": 0.4,  # posible agenda mediática / política
    "C5": 0.2,  # marco realista pobre

    # PRINCIPIO
    "P1": 0.1,  # aumenta entropía
    "P2": 0.2,  # poca coherencia con el todo
    "P3": 0.1,  # ataca dignidad de un grupo
    "P4": 0.1,  # alimenta división/odio
    "P5": 0.5,  # casi no abre acción responsable
}

res_1 = evaluate_with_matrix(claim_1, scores_1)

res_1.F, res_1.C, res_1.P, res_1.D, res_1.action


(0.3,
 0.26,
 0.2,
 0.25333333333333335,
 'Cuestionar, corregir o contrastar fuentes')

In [7]:
# Mostrar la tabla completa de criterios con sus valores
res_1.table


,axis,code,name,question,valor
0,F,F1,Fuente confiable,La fuente es confiable / verificable,0.3
1,F,F2,Consistencia con hechos,La afirmación es consistente con hechos conocidos,0.3
2,F,F3,Funcionamiento real del mundo,Coincide con el funcionamiento real del mundo,0.2
3,F,F4,Sin suposiciones forzadas,No requiere suposiciones forzadas para sostenerse,0.2
4,F,F5,Sin contradicción fuerte,No contradice evidencia sólida,0.5
5,C,C1,Contexto claro,Existe contexto claro y transparente,0.5
6,C,C2,Sin presión emocional/manipulación,"No proviene de emoción, presión o manipulación",0.2
7,C,C3,Sin estigmatizar grupos,No generaliza ni estigmatiza grupos completos,0.0
8,C,C4,Sin agendas interesadas,Está aislada de agendas políticas o mediáticas...,0.4
9,C,C5,Marco realista,Los datos o hechos están situados en un marco ...,0.2


In [8]:
def empty_scores_template() -> Dict[str, float]:
    """
    Devuelve un diccionario {code: 0.5} como punto de partida neutro.
    """
    return {c.code: 0.5 for c in CRITERIA}


template_scores = empty_scores_template()
template_df = criteria_template()
template_df["valor"] = template_df["code"].map(template_scores)
template_df


,axis,code,name,question,valor
0,F,F1,Fuente confiable,La fuente es confiable / verificable,0.5
1,F,F2,Consistencia con hechos,La afirmación es consistente con hechos conocidos,0.5
2,F,F3,Funcionamiento real del mundo,Coincide con el funcionamiento real del mundo,0.5
3,F,F4,Sin suposiciones forzadas,No requiere suposiciones forzadas para sostenerse,0.5
4,F,F5,Sin contradicción fuerte,No contradice evidencia sólida,0.5
5,C,C1,Contexto claro,Existe contexto claro y transparente,0.5
6,C,C2,Sin presión emocional/manipulación,"No proviene de emoción, presión o manipulación",0.5
7,C,C3,Sin estigmatizar grupos,No generaliza ni estigmatiza grupos completos,0.5
8,C,C4,Sin agendas interesadas,Está aislada de agendas políticas o mediáticas...,0.5
9,C,C5,Marco realista,Los datos o hechos están situados en un marco ...,0.5


In [9]:
claim_2 = "Reducir la jornada laboral mejora productividad y salud mental."
res_2 = evaluate_with_matrix(claim_2, template_scores)
res_2.D, res_2.action


(0.5, 'Considerar válido, complementar con más análisis')